# Voting Orientations Analysis

## Overview
This notebook analyzes the **voting orientations** present in the Brazilian legislative voting dataset. Voting orientations indicate the official position or recommendation given to legislators on how to vote for specific propositions.

## Research Questions
1. **Coverage**: How many voting sessions have orientation data vs. missing data?
2. **Government Position**: What is the frequency and pattern of "Governo"/"Gov." orientations?
3. **Orientation Types**: What are all the different types of orientations in the dataset?
4. **Temporal Patterns**: How do orientations change over time and across legislatures?
5. **Approval Correlation**: How do different orientations correlate with final voting outcomes?

## Methodology
- **Data Source**: Brazilian legislative voting records with orientation information
- **Analysis Scope**: Complete temporal coverage across all available legislatures
- **Focus Areas**: Government orientations, coverage analysis, and predictive relationships

Let's begin with a comprehensive exploration of the orientation data.


In [38]:
# INITIAL DATA LOADING AND EXPLORATION
print("=== LOADING AND EXPLORING VOTING ORIENTATIONS DATA ===")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Load orientation data from yearly files
print("📊 Loading orientation data from yearly files...")

# List to hold DataFrames for each year
orientation_dfs = []

# Loop through years 2004 to 2024 (inclusive)
for year in range(2004, 2025):
    file_path = f"../../data/voting/orientations/votacoesOrientacoes-{year}.csv"
    try:
        # The files are semicolon-separated
        df = pd.read_csv(file_path, sep=';')
        df['year'] = year  # Add a year column for reference
        orientation_dfs.append(df)
        print(f"  ✅ Loaded {year}: {len(df):,} rows")
    except FileNotFoundError:
        print(f"  ❌ File not found for year {year}: {file_path}")
    except pd.errors.ParserError as e:
        print(f"  ❌ ParserError for year {year}: {e}. Skipping this file.")

# Concatenate all DataFrames into a single DataFrame
if orientation_dfs:
    orientations_df = pd.concat(orientation_dfs, ignore_index=True)
    print(f"\\n✅ Successfully loaded orientation data!")
    print(f"📊 Total orientation records: {len(orientations_df):,}")
    print(f"📅 Years covered: {orientation_dfs[0]['year'].min() if len(orientation_dfs) > 0 else 'N/A'} to {orientation_dfs[-1]['year'].max() if len(orientation_dfs) > 0 else 'N/A'}")
    print(f"📋 Total columns: {len(orientations_df.columns)}")
    
    # Display column names
    print(f"\\n🔍 Orientation data columns:")
    all_columns = list(orientations_df.columns)
    for i, col in enumerate(all_columns):
        print(f"  {i+1:2d}. {col}")
    
    # Show basic dataset info
    print(f"\\n📊 Orientation dataset info:")
    print(f"  Rows: {len(orientations_df):,}")
    print(f"  Columns: {len(orientations_df.columns)}")
    print(f"  Memory usage: {orientations_df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
    
    # Show sample of orientation data
    print(f"\\n📋 Sample of orientation data (first 3 rows):")
    display(orientations_df.head(3))
    
else:
    print("❌ No orientation data loaded - cannot proceed with analysis")
    orientations_df = None

print("\\n✅ Data loading completed!")


=== LOADING AND EXPLORING VOTING ORIENTATIONS DATA ===
📊 Loading orientation data from yearly files...
  ✅ Loaded 2004: 1,110 rows
  ✅ Loaded 2005: 755 rows
  ✅ Loaded 2006: 915 rows
  ✅ Loaded 2007: 1,259 rows
  ✅ Loaded 2008: 1,251 rows
  ✅ Loaded 2009: 1,876 rows
  ✅ Loaded 2010: 1,003 rows
  ✅ Loaded 2011: 1,283 rows
  ✅ Loaded 2012: 1,057 rows
  ✅ Loaded 2013: 2,403 rows
  ✅ Loaded 2014: 1,429 rows
  ✅ Loaded 2015: 5,018 rows
  ✅ Loaded 2016: 4,895 rows
  ✅ Loaded 2017: 4,865 rows
  ✅ Loaded 2018: 2,935 rows
  ✅ Loaded 2019: 7,085 rows
  ✅ Loaded 2020: 9,352 rows
  ✅ Loaded 2021: 22,415 rows
  ✅ Loaded 2022: 14,415 rows
  ✅ Loaded 2023: 4,359 rows
  ✅ Loaded 2024: 4,361 rows
\n✅ Successfully loaded orientation data!
📊 Total orientation records: 94,041
📅 Years covered: 2004 to 2024
📋 Total columns: 8
\n🔍 Orientation data columns:
   1. idVotacao
   2. uriVotacao
   3. siglaOrgao
   4. descricao
   5. siglaBancada
   6. uriBancada
   7. orientacao
   8. year
\n📊 Orientation dataset 

,idVotacao,uriVotacao,siglaOrgao,descricao,siglaBancada,uriBancada,orientacao,year
0,150399-149,https://dadosabertos.camara.leg.br/api/v2/vota...,PLEN,Rejeitado o destaque.\r\nSim: 114; Não: 235; A...,PPS,https://dadosabertos.camara.leg.br/api/v2/part...,Não,2004
1,150399-149,https://dadosabertos.camara.leg.br/api/v2/vota...,PLEN,Rejeitado o destaque.\r\nSim: 114; Não: 235; A...,PSC,https://dadosabertos.camara.leg.br/api/v2/part...,Não,2004
2,150399-149,https://dadosabertos.camara.leg.br/api/v2/vota...,PLEN,Rejeitado o destaque.\r\nSim: 114; Não: 235; A...,PMDB,https://dadosabertos.camara.leg.br/api/v2/part...,Sim,2004


\n✅ Data loading completed!


In [39]:
# GOVERNMENT ORIENTATION VS VOTING OUTCOMES ANALYSIS
print("=== GOVERNMENT ORIENTATION VS VOTING OUTCOMES ANALYSIS ===")

# Load main voting sessions data for outcome analysis
print("📊 Loading main voting sessions data for outcome analysis...")

try:
    # Try different paths for the main voting dataset
    paths_to_try = [
        "../data/vote_sessions_full.csv",
        "../../data/vote_sessions_full.csv", 
        "../../data/vote_sessions.csv",
        "../data/vote_sessions.csv"
    ]
    
    main_voting_df = None
    for path in paths_to_try:
        try:
            main_voting_df = pd.read_csv(path)
            print(f"✅ Loaded main voting data from: {path}")
            print(f"   Rows: {len(main_voting_df):,}, Columns: {len(main_voting_df.columns)}")
            break
        except FileNotFoundError:
            continue
    
    if main_voting_df is None:
        print("❌ Could not find main voting sessions dataset")
        print("⚠️ Will analyze orientation data only (without voting outcomes)")
        
except Exception as e:
    print(f"❌ Error loading main voting data: {e}")
    main_voting_df = None

if main_voting_df is not None and orientations_df is not None:
    print(f"\\n🔗 LINKING ORIENTATION DATA WITH VOTING OUTCOMES:")
    print("="*60)
    
    # Convert date columns for proper comparison if needed
    if 'data' in main_voting_df.columns:
        main_voting_df['data'] = pd.to_datetime(main_voting_df['data'])
        main_voting_df['year'] = main_voting_df['data'].dt.year
    
    print(f"Main voting data: {len(main_voting_df):,} sessions")
    print(f"Orientation data: {len(orientations_df):,} records")
    
    # Analyze orientation data for patterns - Look for government in siglaBancada or siglaOrgao
    print(f"\\n🎯 Searching for government orientations in siglaBancada and siglaOrgao columns...")
    
    # Check if the target columns exist
    target_columns = ['siglaBancada', 'siglaOrgao']
    available_target_columns = [col for col in target_columns if col in orientations_df.columns]
    
    print(f"   Target columns to check: {target_columns}")
    print(f"   Available target columns: {available_target_columns}")
    
    if available_target_columns:
        # Create working dataset with target columns
        base_cols = ['idVotacao', 'year'] if 'idVotacao' in orientations_df.columns else ['year']
        if 'id' in orientations_df.columns and 'idVotacao' not in orientations_df.columns:
            base_cols = ['id', 'year']
        
        working_cols = [col for col in base_cols if col in orientations_df.columns] + available_target_columns
        gov_orientations = orientations_df[working_cols].copy()
        
        print(f"   Working with {len(gov_orientations):,} records and {len(available_target_columns)} target columns")
        
        # Search for government patterns in the target columns
        gov_patterns = ['governo', 'gov.', 'gov ', 'government']
        government_records = []
        gov_data_summary = {}
        
        for target_col in available_target_columns:
            print(f"\\n   📊 Analyzing column '{target_col}' for government patterns...")
            
            # Get non-null values
            col_data = gov_orientations[target_col].dropna()
            print(f"      Non-null records: {len(col_data):,}")
            
            if len(col_data) > 0:
                # Show sample of values in this column
                value_counts = col_data.value_counts().head(20)
                print(f"      Top 20 values in '{target_col}':")
                for i, (value, count) in enumerate(value_counts.items()):
                    percentage = (count / len(col_data)) * 100
                    print(f"        {i+1:2d}. '{value}': {count:,} ({percentage:.1f}%)")
                
                # Search for government patterns
                gov_mask_col = col_data.astype(str).str.lower().str.contains('|'.join(gov_patterns), na=False)
                gov_records_col = col_data[gov_mask_col]
                
                print(f"      Records with government patterns: {len(gov_records_col):,}")
                
                if len(gov_records_col) > 0:
                    gov_value_counts = gov_records_col.value_counts()
                    print(f"      Government values found:")
                    for value, count in gov_value_counts.items():
                        percentage = (count / len(gov_records_col)) * 100
                        print(f"        '{value}': {count:,} ({percentage:.1f}%)")
                    
                    # Add to government records collection
                    gov_indices = col_data[gov_mask_col].index
                    for idx in gov_indices:
                        government_records.append({
                            'index': idx,
                            'column': target_col,
                            'gov_value': col_data.loc[idx]
                        })
                    
                    gov_data_summary[target_col] = {
                        'total_records': len(col_data),
                        'gov_records': len(gov_records_col),
                        'gov_coverage': len(gov_records_col) / len(col_data) * 100,
                        'unique_gov_values': len(gov_value_counts),
                        'top_gov_value': gov_value_counts.index[0],
                        'top_gov_count': gov_value_counts.iloc[0]
                    }
                else:
                    print(f"      ❌ No government patterns found in '{target_col}'")
                    gov_data_summary[target_col] = {
                        'total_records': len(col_data),
                        'gov_records': 0,
                        'gov_coverage': 0,
                        'unique_gov_values': 0,
                        'top_gov_value': None,
                        'top_gov_count': 0
                    }
            else:
                print(f"      ⚠️ No data in column '{target_col}'")
        
        # Create government mask for the working dataset
        gov_mask = pd.Series(False, index=gov_orientations.index)
        government_columns = available_target_columns  # For compatibility with later code
        
        if government_records:
            # Mark all records that have government orientations
            gov_indices = [rec['index'] for rec in government_records]
            gov_mask.loc[gov_indices] = True
            
            print(f"\\n✅ Found {len(government_records):,} government orientation records across {len(available_target_columns)} columns")
            print(f"   Unique sessions with government orientations: {gov_mask.sum():,}")
            
            # Summary by column
            print(f"\\n📊 Government orientation summary by column:")
            for col, summary in gov_data_summary.items():
                if summary['gov_records'] > 0:
                    print(f"   {col}: {summary['gov_records']:,} records ({summary['gov_coverage']:.1f}% coverage)")
                    print(f"     Top government value: '{summary['top_gov_value']}' ({summary['top_gov_count']:,} times)")
        else:
            print(f"\\n❌ No government orientations found in any target columns")
        
        # Analysis by year to compare with voting sessions
        yearly_voting_sessions = main_voting_df.groupby('year').size() if 'year' in main_voting_df.columns else pd.Series()
        yearly_orientations = gov_orientations.groupby('year').size() if 'year' in gov_orientations.columns else pd.Series()
        yearly_gov_orientations = gov_orientations[gov_mask].groupby('year').size() if 'year' in gov_orientations.columns and gov_mask.any() else pd.Series()
        
    else:
        print(f"❌ Target columns {target_columns} not found in orientation dataset")
        print(f"💡 Available columns: {list(orientations_df.columns)[:30]}")
        
        # Set empty defaults
        gov_orientations = pd.DataFrame()
        government_columns = []
        gov_data_summary = {}
        yearly_voting_sessions = pd.Series()
        yearly_orientations = pd.Series()
        yearly_gov_orientations = pd.Series()
        gov_mask = pd.Series(dtype=bool)
            
    # Continue with coverage analysis if we have government data
    if government_columns and len(gov_orientations) > 0:
        print(f"\\n📊 GOVERNMENT ORIENTATION COVERAGE ANALYSIS:")
        print("-" * 50)
        
        print("Year | Vote Sessions | Orientations | Gov Orientations | Gov Coverage %")
        print("-" * 75)
        
        total_vote_sessions = 0
        total_orientations = 0
        total_gov_orientations = 0
        
        if len(yearly_voting_sessions) > 0:
            all_years = sorted(set(yearly_voting_sessions.index) | set(yearly_orientations.index))
        else:
            all_years = sorted(yearly_orientations.index) if len(yearly_orientations) > 0 else []
        
        for year in all_years:
            vote_sessions = yearly_voting_sessions.get(year, 0)
            orientations = yearly_orientations.get(year, 0)
            gov_orientations_count = yearly_gov_orientations.get(year, 0)
            
            if orientations > 0:
                gov_coverage = (gov_orientations_count / orientations) * 100
            else:
                gov_coverage = 0
            
            print(f"{year} | {vote_sessions:12,} | {orientations:11,} | {gov_orientations_count:15,} | {gov_coverage:10.1f}%")
            
            total_vote_sessions += vote_sessions
            total_orientations += orientations
            total_gov_orientations += gov_orientations_count
        
        overall_gov_coverage = (total_gov_orientations / total_orientations * 100) if total_orientations > 0 else 0
        
        print("-" * 75)
        print(f"Total| {total_vote_sessions:12,} | {total_orientations:11,} | {total_gov_orientations:15,} | {overall_gov_coverage:10.1f}%")
            
        # Government ACTUAL orientation values from 'orientacao' column for Governo/Gov. records
        print(f"\\n📈 ACTUAL GOVERNMENT ORIENTATION VALUES (from 'orientacao' column):")
        print("-" * 70)
        
        # Filter records where siglaBancada contains "Governo" or "Gov."
        if 'siglaBancada' in orientations_df.columns and 'orientacao' in orientations_df.columns:
            # Get government records (Governo or Gov. in siglaBancada)
            gov_patterns = ['governo', 'gov.', 'gov ', 'government']
            gov_bancada_mask = orientations_df['siglaBancada'].astype(str).str.lower().str.contains('|'.join(gov_patterns), na=False)
            
            government_orientation_records = orientations_df[gov_bancada_mask].copy()
            
            print(f"📊 Found {len(government_orientation_records):,} records with government bancada")
            
            if len(government_orientation_records) > 0:
                print(f"\\n🏛️ Government bancada values found:")
                bancada_counts = government_orientation_records['siglaBancada'].value_counts()
                for value, count in bancada_counts.items():
                    percentage = (count / len(government_orientation_records)) * 100
                    print(f"   '{value}': {count:,} ({percentage:.1f}%)")
                
                # Now analyze the actual orientations for these government records
                gov_orientations_with_values = government_orientation_records['orientacao'].dropna()
                
                print(f"\\n🎯 ACTUAL GOVERNMENT ORIENTATION VALUES (from 'orientacao' column):")
                print(f"   Records with orientation values: {len(gov_orientations_with_values):,}")
                print(f"   Records missing orientation: {len(government_orientation_records) - len(gov_orientations_with_values):,}")
                
                if len(gov_orientations_with_values) > 0:
                    orientation_counts = gov_orientations_with_values.value_counts()
                    
                    print(f"\\n📋 Distribution of government orientation values:")
                    print("Orientation Value | Count | Percentage")
                    print("-" * 45)
                    
                    for i, (orientation, count) in enumerate(orientation_counts.items()):
                        percentage = (count / len(gov_orientations_with_values)) * 100
                        print(f"{str(orientation)[:15]:15} | {count:5,} | {percentage:8.1f}%")
                    
                    # Summary statistics
                    print(f"\\n📊 Government Orientation Summary:")
                    print(f"  Total government records: {len(government_orientation_records):,}")
                    print(f"  Records with orientation data: {len(gov_orientations_with_values):,}")
                    print(f"  Unique orientation values: {len(orientation_counts)}")
                    print(f"  Data completeness: {len(gov_orientations_with_values)/len(government_orientation_records)*100:.1f}%")
                    
                    # Most common orientations
                    print(f"\\n🥇 Top government orientation values:")
                    for i, (orientation, count) in enumerate(orientation_counts.head(10).items()):
                        percentage = (count / len(gov_orientations_with_values)) * 100
                        print(f"  {i+1:2d}. '{orientation}': {count:,} ({percentage:.1f}%)")
                    
                    # Analysis by government bancada type
                    print(f"\\n🔍 Orientation breakdown by government bancada:")
                    for bancada_value in bancada_counts.index:
                        bancada_records = government_orientation_records[government_orientation_records['siglaBancada'] == bancada_value]
                        bancada_orientations = bancada_records['orientacao'].dropna()
                        
                        if len(bancada_orientations) > 0:
                            print(f"\\n   📋 '{bancada_value}' ({len(bancada_records):,} records):")
                            bancada_orientation_counts = bancada_orientations.value_counts()
                            
                            for orientation, count in bancada_orientation_counts.head(5).items():
                                percentage = (count / len(bancada_orientations)) * 100
                                print(f"      '{orientation}': {count:,} ({percentage:.1f}%)")
                            
                            if len(bancada_orientation_counts) > 5:
                                remaining = len(bancada_orientation_counts) - 5
                                print(f"      ... and {remaining} more orientation types")
                    
                    # Temporal analysis
                    if 'year' in government_orientation_records.columns:
                        print(f"\\n📅 Government orientations by year:")
                        yearly_gov_counts = government_orientation_records.groupby('year')['orientacao'].count()
                        
                        print("Year | Gov Records | Sample Orientations")
                        print("-" * 50)
                        
                        for year in sorted(yearly_gov_counts.index):
                            year_records = government_orientation_records[government_orientation_records['year'] == year]
                            year_count = len(year_records)
                            
                            # Get top 3 orientations for this year
                            year_orientations = year_records['orientacao'].dropna().value_counts().head(3)
                            sample_orientations = ", ".join([f"'{k}' ({v})" for k, v in year_orientations.items()])
                            
                            print(f"{year} | {year_count:11,} | {sample_orientations}")
                
                else:
                    print(f"❌ No orientation values found for government records")
            
            else:
                print(f"❌ No government bancada records found")
        
        else:
            print(f"❌ Required columns ('siglaBancada' or 'orientacao') not found in dataset")
            print(f"   Available columns: {list(orientations_df.columns)}")
            
        # Try to link with voting outcomes if we have approval data - ONLY for government orientations
        if 'aprovacao' in main_voting_df.columns and government_columns:
            print(f"\\n🗳️ GOVERNMENT ORIENTATION (Governo/Gov. columns) VS VOTING OUTCOMES:")
            print("-" * 70)
            
            # Find the linking column (idVotacao or similar)
            linking_col = None
            for col in ['idVotacao', 'id']:
                if col in main_voting_df.columns and col in gov_orientations.columns:
                    linking_col = col
                    break
            
            if linking_col and len(gov_orientations) > 0:
                print(f"🔗 Linking datasets using column: '{linking_col}'")
                
                # Prepare data for linking - create a long format with all government orientations
                gov_records_for_linking = []
                
                for gov_col in government_columns:
                    if gov_col in gov_orientations.columns:
                        # Get records with data in this government column
                        col_data = gov_orientations[[linking_col, 'year', gov_col]].dropna()
                        if len(col_data) > 0:
                            col_data = col_data.copy()
                            col_data['gov_column'] = gov_col
                            col_data['gov_orientation'] = col_data[gov_col]
                            col_data = col_data[[linking_col, 'year', 'gov_column', 'gov_orientation']]
                            gov_records_for_linking.append(col_data)
                
                if gov_records_for_linking:
                    # Combine all government orientation records
                    combined_gov_data = pd.concat(gov_records_for_linking, ignore_index=True)
                    print(f"   Prepared {len(combined_gov_data):,} government orientation records for linking")
                    
                    try:
                        # Merge with voting outcomes
                        merged_data = main_voting_df.merge(
                            combined_gov_data,
                            on=linking_col,
                            how='inner'
                        )
                        
                        if len(merged_data) > 0:
                            print(f"✅ Successfully linked {len(merged_data):,} sessions with government orientations")
                            
                            # Analyze approval rates by government orientation value
                            approval_by_orientation = merged_data.groupby('gov_orientation')['aprovacao'].agg([
                                'count', 'mean', 'std'
                            ]).round(4)
                            
                            approval_by_orientation.columns = ['sessions', 'approval_rate', 'approval_std']
                            approval_by_orientation = approval_by_orientation.sort_values('approval_rate', ascending=False)
                            
                            print(f"\\nApproval rates by government orientation value:")
                            print("Government Orientation | Sessions | Approval Rate | Std Dev")
                            print("-" * 70)
                            
                            for orientation, row in approval_by_orientation.iterrows():
                                sessions = int(row['sessions'])
                                approval_rate = row['approval_rate']
                                approval_std = row['approval_std']
                                
                                if sessions >= 5:  # Show orientations with reasonable sample size
                                    print(f"{str(orientation)[:20]:20} | {sessions:8,} | {approval_rate:13.3f} | {approval_std:7.3f}")
                            
                            # Overall statistics for government orientations
                            total_linked_sessions = len(merged_data)
                            overall_approval_rate = merged_data['aprovacao'].mean()
                            
                            print(f"\\nOverall statistics for government-oriented sessions:")
                            print(f"  Total sessions with government orientations: {total_linked_sessions:,}")
                            print(f"  Government sessions approval rate: {overall_approval_rate:.3f}")
                            
                            # Compare with general approval rate
                            general_approval_rate = main_voting_df['aprovacao'].mean()
                            print(f"  General approval rate (all sessions): {general_approval_rate:.3f}")
                            
                            difference = overall_approval_rate - general_approval_rate
                            print(f"  Difference: {difference:+.3f}")
                            
                            if abs(difference) > 0.05:
                                direction = "higher" if difference > 0 else "lower"
                                print(f"  📊 Government-oriented sessions have {direction} approval rates")
                            else:
                                print(f"  📊 Government-oriented sessions have similar approval rates")
                            
                            # Additional analysis of government orientation effectiveness
                            if len(approval_by_orientation) > 1:
                                max_approval = approval_by_orientation['approval_rate'].max()
                                min_approval = approval_by_orientation['approval_rate'].min()
                                print(f"\\n🎯 Government Orientation Effectiveness:")
                                print(f"  Highest approval rate: {max_approval:.3f}")
                                print(f"  Lowest approval rate: {min_approval:.3f}")
                                print(f"  Range: {max_approval - min_approval:.3f}")
                            
                            # Analysis by government column
                            column_analysis = merged_data.groupby('gov_column')['aprovacao'].agg([
                                'count', 'mean'
                            ]).round(3)
                            column_analysis.columns = ['sessions', 'approval_rate']
                            
                            print(f"\\n📊 Analysis by government column:")
                            print("Government Column | Sessions | Approval Rate")
                            print("-" * 45)
                            for col, row in column_analysis.iterrows():
                                sessions = int(row['sessions'])
                                approval_rate = row['approval_rate']
                                print(f"{str(col)[:15]:15} | {sessions:8,} | {approval_rate:13.3f}")
                        
                        else:
                            print("❌ No government-oriented sessions could be linked between datasets")
                            print("💡 This suggests different ID systems or data structure")
                    
                    except Exception as e:
                        print(f"❌ Error linking government orientation datasets: {e}")
                        import traceback
                        print(traceback.format_exc())
                
                else:
                    print("❌ No government orientation data prepared for linking")
            
            else:
                if not linking_col:
                    print("❌ No suitable linking column found between datasets")
                    print(f"   Main voting columns: {list(main_voting_df.columns)[:10]}")
                    print(f"   Orientation columns: {list(gov_orientations.columns)[:10]}")
                else:
                    print("❌ No government orientation data available for outcome analysis")
        else:
            print("❌ Cannot link government orientations with voting outcomes")
            if 'aprovacao' not in main_voting_df.columns:
                print("   Missing 'aprovacao' column in main voting data")
            if not government_columns:
                print("   No government orientation columns found")
    
    else:
        print("❌ No government data available for coverage analysis")

else:
    print("⚠️ Cannot perform outcome analysis without both datasets")

print(f"\\n✅ Government orientation vs voting outcomes analysis completed!")
print("\\n" + "="*80)


=== GOVERNMENT ORIENTATION VS VOTING OUTCOMES ANALYSIS ===
📊 Loading main voting sessions data for outcome analysis...
✅ Loaded main voting data from: ../data/vote_sessions_full.csv
   Rows: 41,461, Columns: 171
\n🔗 LINKING ORIENTATION DATA WITH VOTING OUTCOMES:
Main voting data: 41,461 sessions
Orientation data: 94,041 records
\n🎯 Searching for government orientations in siglaBancada and siglaOrgao columns...
   Target columns to check: ['siglaBancada', 'siglaOrgao']
   Available target columns: ['siglaBancada', 'siglaOrgao']
   Working with 94,041 records and 2 target columns
\n   📊 Analyzing column 'siglaBancada' for government patterns...
      Non-null records: 94,041
      Top 20 values in 'siglaBancada':
         1. 'PT': 4,098 (4.4%)
         2. 'PSDB': 3,862 (4.1%)
         3. 'PSOL': 3,811 (4.1%)
         4. 'PDT': 3,771 (4.0%)
         5. 'PSB': 3,763 (4.0%)
         6. 'Minoria': 3,735 (4.0%)
         7. 'PSD': 3,348 (3.6%)
         8. 'PCdoB': 3,292 (3.5%)
         9. 'PV'

In [40]:
# GOVERNMENT ORIENTATION PREDICTION ANALYSIS
print("=== ANALYZING FEATURES FOR GOVERNMENT ORIENTATION PREDICTION ===")

# Filter government orientation records for modeling
if 'siglaBancada' in orientations_df.columns and 'orientacao' in orientations_df.columns:
    # Get government records with orientation data
    gov_patterns = ['governo', 'gov.', 'gov ', 'government']
    gov_bancada_mask = orientations_df['siglaBancada'].astype(str).str.lower().str.contains('|'.join(gov_patterns), na=False)
    
    government_records = orientations_df[gov_bancada_mask & orientations_df['orientacao'].notna()].copy()
    
    print(f"📊 Government records with orientation data: {len(government_records):,}")
    
    if len(government_records) > 0 and main_voting_df is not None:
        print(f"🔗 Merging with main voting dataset for feature analysis...")
        
        # Merge with main voting data to get features
        merged_gov_data = government_records.merge(
            main_voting_df, 
            on='idVotacao', 
            how='inner'
        )
        
        print(f"✅ Successfully merged {len(merged_gov_data):,} government records with voting features")
        
        if len(merged_gov_data) > 0:
            # Analyze government orientation distribution
            print(f"\\n🎯 GOVERNMENT ORIENTATION DISTRIBUTION:")
            orientation_dist = merged_gov_data['orientacao'].value_counts()
            
            print("Orientation | Count | Percentage")
            print("-" * 35)
            for orientation, count in orientation_dist.items():
                percentage = (count / len(merged_gov_data)) * 100
                print(f"{str(orientation)[:10]:10} | {count:5,} | {percentage:7.1f}%")
            
            # Focus on main orientations (Sim/Não) for prediction
            main_orientations = ['Sim', 'Não']
            available_main_orientations = [o for o in main_orientations if o in orientation_dist.index]
            
            if len(available_main_orientations) >= 2:
                print(f"\\n🎯 Focusing on main orientations for prediction: {available_main_orientations}")
                
                # Filter to main orientations for modeling
                modeling_data = merged_gov_data[merged_gov_data['orientacao'].isin(available_main_orientations)].copy()
                print(f"   Records for modeling: {len(modeling_data):,}")
                
                # FEATURE ANALYSIS
                print(f"\\n📋 FEATURE CORRELATION ANALYSIS:")
                print("="*60)
                
                # Available features for analysis
                potential_features = [
                    'idDeputadoAutor', 'nomeDeputadoAutor', 'siglaPartidoAutor', 
                    'siglaUfAutor', 'legislatura', 'ano', 'mes', 
                    'tipoProposicao', 'numeroProposicao'
                ]
                
                available_features = [f for f in potential_features if f in modeling_data.columns]
                print(f"Available features for analysis: {available_features}")
                
                # 1. PARTY ANALYSIS
                if 'siglaPartidoAutor' in available_features:
                    print(f"\\n🏛️ PARTY vs GOVERNMENT ORIENTATION:")
                    party_orientation = pd.crosstab(
                        modeling_data['siglaPartidoAutor'], 
                        modeling_data['orientacao'], 
                        normalize='index'
                    ).round(3)
                    
                    print("\\nTop parties by 'Sim' orientation rate:")
                    if 'Sim' in party_orientation.columns:
                        party_sim_rate = party_orientation['Sim'].sort_values(ascending=False)
                        for i, (party, rate) in enumerate(party_sim_rate.head(10).items()):
                            party_count = modeling_data[modeling_data['siglaPartidoAutor'] == party].shape[0]
                            if party_count >= 5:  # Only show parties with reasonable sample size
                                print(f"  {i+1:2d}. {party}: {rate:.1%} ({party_count:,} records)")
                    
                    print("\\nTop parties by 'Não' orientation rate:")
                    if 'Não' in party_orientation.columns:
                        party_nao_rate = party_orientation['Não'].sort_values(ascending=False)
                        for i, (party, rate) in enumerate(party_nao_rate.head(10).items()):
                            party_count = modeling_data[modeling_data['siglaPartidoAutor'] == party].shape[0]
                            if party_count >= 5:
                                print(f"  {i+1:2d}. {party}: {rate:.1%} ({party_count:,} records)")
                
                # 2. TEMPORAL ANALYSIS
                if 'ano' in available_features:
                    print(f"\\n📅 TEMPORAL PATTERNS:")
                    temporal_orientation = pd.crosstab(
                        modeling_data['ano'], 
                        modeling_data['orientacao'], 
                        normalize='index'
                    ).round(3)
                    
                    print("\\nGovernment orientation trends by year:")
                    print("Year | Sim Rate | Não Rate | Total Records")
                    print("-" * 45)
                    
                    for year in sorted(temporal_orientation.index):
                        sim_rate = temporal_orientation.loc[year, 'Sim'] if 'Sim' in temporal_orientation.columns else 0
                        nao_rate = temporal_orientation.loc[year, 'Não'] if 'Não' in temporal_orientation.columns else 0
                        year_count = modeling_data[modeling_data['ano'] == year].shape[0]
                        
                        print(f"{year} | {sim_rate:7.1%} | {nao_rate:8.1%} | {year_count:12,}")
                
                # 3. PROPOSITION TYPE ANALYSIS
                if 'tipoProposicao' in available_features:
                    print(f"\\n📜 PROPOSITION TYPE vs ORIENTATION:")
                    prop_type_orientation = pd.crosstab(
                        modeling_data['tipoProposicao'], 
                        modeling_data['orientacao'], 
                        normalize='index'
                    ).round(3)
                    
                    print("\\nProposition types by 'Sim' orientation rate:")
                    if 'Sim' in prop_type_orientation.columns:
                        prop_sim_rate = prop_type_orientation['Sim'].sort_values(ascending=False)
                        for i, (prop_type, rate) in enumerate(prop_sim_rate.head(10).items()):
                            prop_count = modeling_data[modeling_data['tipoProposicao'] == prop_type].shape[0]
                            if prop_count >= 3:
                                print(f"  {i+1:2d}. {prop_type}: {rate:.1%} ({prop_count:,} records)")
                
                # 4. REGIONAL ANALYSIS
                if 'siglaUfAutor' in available_features:
                    print(f"\\n🗺️ REGIONAL PATTERNS:")
                    uf_orientation = pd.crosstab(
                        modeling_data['siglaUfAutor'], 
                        modeling_data['orientacao'], 
                        normalize='index'
                    ).round(3)
                    
                    print("\\nStates by 'Sim' orientation rate:")
                    if 'Sim' in uf_orientation.columns:
                        uf_sim_rate = uf_orientation['Sim'].sort_values(ascending=False)
                        for i, (uf, rate) in enumerate(uf_sim_rate.head(10).items()):
                            uf_count = modeling_data[modeling_data['siglaUfAutor'] == uf].shape[0]
                            if uf_count >= 5:
                                print(f"  {i+1:2d}. {uf}: {rate:.1%} ({uf_count:,} records)")
                
                # 5. PREDICTIVE MODELING
                print(f"\\n🤖 BUILDING PREDICTIVE MODEL:")
                print("="*50)
                
                # Prepare features for modeling
                from sklearn.model_selection import train_test_split
                from sklearn.ensemble import RandomForestClassifier
                from sklearn.preprocessing import LabelEncoder
                from sklearn.metrics import classification_report, confusion_matrix
                
                # Select features for modeling
                modeling_features = []
                feature_encoders = {}
                
                for feature in ['siglaPartidoAutor', 'siglaUfAutor', 'tipoProposicao', 'ano', 'legislatura']:
                    if feature in modeling_data.columns:
                        # Handle missing values
                        modeling_data[feature] = modeling_data[feature].fillna('Unknown')
                        
                        # Encode categorical features
                        if modeling_data[feature].dtype == 'object':
                            le = LabelEncoder()
                            modeling_data[f'{feature}_encoded'] = le.fit_transform(modeling_data[feature].astype(str))
                            feature_encoders[feature] = le
                            modeling_features.append(f'{feature}_encoded')
                        else:
                            modeling_features.append(feature)
                
                if len(modeling_features) > 0:
                    print(f"Features for modeling: {[f.replace('_encoded', '') for f in modeling_features]}")
                    
                    # Prepare data
                    X = modeling_data[modeling_features]
                    y = modeling_data['orientacao']
                    
                    # Split data
                    X_train, X_test, y_train, y_test = train_test_split(
                        X, y, test_size=0.2, random_state=42, stratify=y
                    )
                    
                    print(f"\\nTraining samples: {len(X_train):,}")
                    print(f"Testing samples: {len(X_test):,}")
                    
                    # Train model
                    rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
                    rf_model.fit(X_train, y_train)
                    
                    # Make predictions
                    y_pred = rf_model.predict(X_test)
                    
                    # Evaluate model
                    print(f"\\n📊 MODEL PERFORMANCE:")
                    print("\\nClassification Report:")
                    print(classification_report(y_test, y_pred))
                    
                    # Feature importance
                    feature_importance = pd.DataFrame({
                        'feature': [f.replace('_encoded', '') for f in modeling_features],
                        'importance': rf_model.feature_importances_
                    }).sort_values('importance', ascending=False)
                    
                    print(f"\\n🎯 FEATURE IMPORTANCE:")
                    for _, row in feature_importance.iterrows():
                        print(f"  {row['feature']}: {row['importance']:.3f}")
                    
                    # APPLY MODEL TO PREDICT MISSING ORIENTATIONS
                    print(f"\\n🔮 PREDICTING MISSING GOVERNMENT ORIENTATIONS:")
                    print("="*55)
                    
                    # Find sessions without government orientation
                    sessions_without_gov_orientation = main_voting_df[
                        ~main_voting_df['idVotacao'].isin(government_records['idVotacao'])
                    ].copy()
                    
                    print(f"Sessions without government orientation: {len(sessions_without_gov_orientation):,}")
                    
                    # Prepare features for prediction
                    predict_data = sessions_without_gov_orientation.copy()
                    prediction_features = []
                    
                    for feature in ['siglaPartidoAutor', 'siglaUfAutor', 'tipoProposicao', 'ano', 'legislatura']:
                        if feature in predict_data.columns and feature in feature_encoders:
                            # Fill missing values
                            predict_data[feature] = predict_data[feature].fillna('Unknown')
                            
                            # Handle unseen categories
                            le = feature_encoders[feature]
                            predict_data[f'{feature}_encoded'] = predict_data[feature].astype(str).apply(
                                lambda x: le.transform([x])[0] if x in le.classes_ else -1
                            )
                            prediction_features.append(f'{feature}_encoded')
                        elif feature in predict_data.columns and f'{feature}_encoded' in modeling_features:
                            prediction_features.append(feature)
                    
                    if len(prediction_features) > 0:
                        # Make predictions
                        X_predict = predict_data[prediction_features]
                        predicted_orientations = rf_model.predict(X_predict)
                        predicted_probabilities = rf_model.predict_proba(X_predict)
                        
                        # Add predictions to data
                        predict_data['predicted_gov_orientation'] = predicted_orientations
                        predict_data['prediction_confidence'] = predicted_probabilities.max(axis=1)
                        
                        # Summary of predictions
                        prediction_summary = pd.Series(predicted_orientations).value_counts()
                        
                        print(f"\\n📈 PREDICTION SUMMARY:")
                        print("Predicted Orientation | Count | Percentage")
                        print("-" * 45)
                        for orientation, count in prediction_summary.items():
                            percentage = (count / len(predicted_orientations)) * 100
                            print(f"{str(orientation)[:18]:18} | {count:5,} | {percentage:7.1f}%")
                        
                        # High confidence predictions
                        high_confidence = predict_data[predict_data['prediction_confidence'] >= 0.8]
                        print(f"\\nHigh confidence predictions (≥80%): {len(high_confidence):,}")
                        
                        if len(high_confidence) > 0:
                            print("\\nSample high-confidence predictions:")
                            sample_predictions = high_confidence[['idVotacao', 'nomeDeputadoAutor', 'siglaPartidoAutor', 
                                                               'tipoProposicao', 'predicted_gov_orientation', 
                                                               'prediction_confidence']].head(10)
                            
                            for _, row in sample_predictions.iterrows():
                                print(f"  Session {row['idVotacao']}: {row['predicted_gov_orientation']} "
                                     f"({row['prediction_confidence']:.1%} confidence)")
                                print(f"    Author: {row['nomeDeputadoAutor']} ({row['siglaPartidoAutor']}) - {row['tipoProposicao']}")
                        
                        # Save predictions for future use
                        print(f"\\n💾 Saving predictions for {len(predict_data):,} sessions...")
                        predict_data[['idVotacao', 'predicted_gov_orientation', 'prediction_confidence']].to_csv(
                            'government_orientation_predictions.csv', index=False
                        )
                        print(f"✅ Predictions saved to 'government_orientation_predictions.csv'")
                    
                    else:
                        print("❌ No suitable features available for prediction")
                
                else:
                    print("❌ No features available for modeling")
            
            else:
                print(f"❌ Insufficient main orientations for prediction modeling")
                print(f"   Available orientations: {list(orientation_dist.index)}")
        
        else:
            print("❌ No records after merging with main voting dataset")
    
    else:
        print("❌ No government records with orientation data or main voting dataset not available")

else:
    print("❌ Required columns not found for government orientation analysis")

print(f"\\n✅ Government orientation prediction analysis completed!")
print("\\n" + "="*80)


=== ANALYZING FEATURES FOR GOVERNMENT ORIENTATION PREDICTION ===
📊 Government records with orientation data: 4,034
🔗 Merging with main voting dataset for feature analysis...
✅ Successfully merged 6,594 government records with voting features
\n🎯 GOVERNMENT ORIENTATION DISTRIBUTION:
Orientation | Count | Percentage
-----------------------------------
Sim        | 3,587 |    54.4%
Não        | 2,724 |    41.3%
Liberado   |   283 |     4.3%
\n🎯 Focusing on main orientations for prediction: ['Sim', 'Não']
   Records for modeling: 6,311
\n📋 FEATURE CORRELATION ANALYSIS:
Available features for analysis: ['idDeputadoAutor', 'legislatura']
\n🤖 BUILDING PREDICTIVE MODEL:
Features for modeling: ['legislatura']
\nTraining samples: 5,048
Testing samples: 1,263
\n📊 MODEL PERFORMANCE:
\nClassification Report:
              precision    recall  f1-score   support

         Não       0.48      0.68      0.56       545
         Sim       0.65      0.45      0.53       718

    accuracy                 